In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup

# Chemin vers le fichier CSV contenant les restaurants
dataset_path = "restaurants_paris_with_url.csv"  
output_path = "restaurants_google_with_reviews.csv"  

# Charger le dataset
restaurants_df = pd.read_csv(dataset_path)

# Supprimer les doublons
restaurants_df = restaurants_df.drop_duplicates(keep="first").reset_index(drop=True)

# Ajouter une colonne pour stocker les URL des avis et les avis eux-mêmes
restaurants_df["reviews_url"] = None
restaurants_df["reviews"] = None

# Configurer le chemin du ChromeDriver
chromedriver_path = '/Users/stella/Desktop/A5/WEBSCRAPPING/chromedriver-mac-x64/chromedriver'

# Définir les fonctions existantes
def get_reviews_page_url(url):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=Service(chromedriver_path), options=options)
    driver.get(url)
    time.sleep(5)

    # Gérer le bouton "Tout refuser" pour les cookies
    try:
        cookies_button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[1]/div/div/button'))
        )
        cookies_button.click()
        print("Cookies refusés avec succès!")
    except Exception as e:
        print("Erreur lors de la gestion des cookies :", e)

    # Cliquer sur le bouton des avis
    try:
        button = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]'))
        )
        button.click()
        print("Bouton des avis cliqué avec succès!")
    except Exception as e:
        print("Erreur lors de l'interaction avec le bouton :", e)
        driver.quit()
        return None

    # Attendre que l'URL change après avoir cliqué
    WebDriverWait(driver, 20).until(
        lambda driver: driver.current_url != url
    )
    reviews_url = driver.current_url
    driver.quit()
    return reviews_url

def scrape_first_5_reviews(url):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=Service(chromedriver_path), options=options)
    driver.get(url)
    time.sleep(5)

    # Gérer le bouton "Tout refuser" pour les cookies
    try:
        cookies_button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[1]/div/div/button'))
        )
        cookies_button.click()
        print("Cookies refusés avec succès!")
    except Exception as e:
        print("Erreur lors de la gestion des cookies :", e)

    time.sleep(5)
    page_source = driver.page_source

    # Utiliser BeautifulSoup pour extraire les avis
    soup = BeautifulSoup(page_source, 'html.parser')
    review_elements = soup.find_all('span', class_='wiI7pd')

    reviews = [review.get_text() for review in review_elements[:5]]
    driver.quit()
    return reviews

# Boucle sur toutes les lignes du dataset
for index, row in restaurants_df.iterrows():
    print(f"Traitement du restaurant {index + 1}/{len(restaurants_df)} : {row['name']}")
    try:
        # Récupérer l'URL des avis
        reviews_url = get_reviews_page_url(row["google_maps_url"])
        restaurants_df.at[index, "reviews_url"] = reviews_url

        # Récupérer les 5 premiers avis
        if reviews_url:
            reviews = scrape_first_5_reviews(reviews_url)
            restaurants_df.at[index, "reviews"] = reviews
        else:
            print(f"Impossible de récupérer l'URL des avis pour {row['name']}")
    except Exception as e:
        print(f"Erreur lors du traitement du restaurant {row['name']} : {e}")

# Sauvegarder les résultats dans un fichier CSV
restaurants_df.to_csv(output_path, index=False)
print(f"Scraping terminé. Résultats enregistrés dans {output_path}")


Traitement du restaurant 1/380 : L'Aubergeade
Cookies refusés avec succès!
Bouton des avis cliqué avec succès!
Cookies refusés avec succès!
Traitement du restaurant 2/380 : La Cantine de daumesnil - restaurant paris 12
Cookies refusés avec succès!
Bouton des avis cliqué avec succès!
Cookies refusés avec succès!
Traitement du restaurant 3/380 : Le Picotin
Cookies refusés avec succès!
Bouton des avis cliqué avec succès!
Cookies refusés avec succès!
Traitement du restaurant 4/380 : L'Alchimiste
Cookies refusés avec succès!
Bouton des avis cliqué avec succès!
Cookies refusés avec succès!
Traitement du restaurant 5/380 : Ty Louis - Restaurant Breton Paris 12
Cookies refusés avec succès!
Bouton des avis cliqué avec succès!
Cookies refusés avec succès!
Traitement du restaurant 6/380 : Les affranchis
Cookies refusés avec succès!
Bouton des avis cliqué avec succès!
Cookies refusés avec succès!
Traitement du restaurant 7/380 : L'Ébauchoir
Cookies refusés avec succès!
Bouton des avis cliqué avec 